In [2]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler #para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test

# SOFTMAX

#### Manual

In [4]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=0)

x = np.array([2.0,1.0,0.1])
outputs = softmax(x)
print('softmax numpy:', outputs)
# Observar q el mayor valor es el de mayor probabilidad

softmax numpy: [0.65900114 0.24243297 0.09856589]


#### Pytorch
Los resultados seran practicamente iguales

In [7]:
x = torch.tensor([2.0,1.0,0.1])
outputs = torch.softmax(x, dim=0) # DEBEMOS ESPECIFICAR LAS DIMENSIONES
print('softmax numpy:', outputs)

softmax numpy: tensor([0.6590, 0.2424, 0.0986])


# CROSS ENTROPY
Mide el rendimneot de modelo de clasificacion cuya salida esta entre 0 y 1. Se puede combinar con softmax.
A medida que la probabilidad predicha diverge de lo real, significa q este valor es muy alto. Por tanto se espera que una buena prediccion tenga una cross entropy bajo.
Utiliza vectores one-hot 

In [8]:
def cross_entropy(actual, predicted):
    loss = -np.sum(actual*np.log(predicted)) #SUMA SOBRE EL PRODUCTO DE LOS TARGETS REALES * PREDECIDOS
    return loss #/ flaot(predicted.shape[0]) #en caso de no ser unidimensional
# y debe ser un vector one hot
# si es clase 0: [1 0 0]
# si es clase 1: [0 1 0]
# si es clase 2: [0 0 1]
Y = np.array([1,0,0])

Y_pred_good = np.array([0.7,0.2,0.1])#SEGUN ESTA PREDICCION
                                    #lA CLASE 1 ES BASTANTE LA PROBABILIDAD, POR TANTO MAS CERCA Y MENOS 
                                    # ENTROPIA CRUZADA, Y SERA BAJA LA PERDIDA
Y_pred_bad = np.array([0.1,0.3,0.6]) #SEGUN ESTA PREDICCION
                                    #lA CLASE 0 ES MUY BAJA LA PROBABILIDAD SIENDO Q ES CORRECTA
                                    # LA ENTROPIA CRUZADA AUMENTA! MUCHA PERDIDA!
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)

print(f'Loss 1 numpy: {l1:.4f}')
print(f'Loss 2 numpy: {l2:.4f}')


Loss 1 numpy: 0.3567
Loss 2 numpy: 2.3026


#### Pytorch
Los resultados seran igual de correcto aunque pueden variar
Pero OJO, el metodo de aqui ya aplica una funcion de softmax y tambn el negativa, por lo tanto hay que tener consideraciones.
* LOS Y TARGETS TIENEN CLASS LABELS = NO USAR ONE HOT, solo  colocar la label correcta
* SI LAS Y PREDICCIONES EN "RAW" (LOGITS), EN BRUTO... = NO  HAY QUE APLICAR SOFTMAX

In [10]:
loss = nn.CrossEntropyLoss()
Y = torch.tensor([0]) #Solo colocamos el class label, NO ONE HOT
#nsamples x nclasses = 1x3
Y_pred_good = torch.tensor([[2.0,1.0,0.1]]) # OJO VALORES EN BRUTOS, SIN SOFTMAX
# Como la clase 0 es mas alta, es una buena prediccion
Y_pred_bad = torch.tensor([[0.5,2.0,0.3]]) #Como la clase 0 es mas baja, es una MALA prediccion

l1 = loss(Y_pred_good,Y)
l2 = loss(Y_pred_bad,Y)

print(f'Loss 1 numpy: {l1.item():.4f}')
print(f'Loss 2 numpy: {l2.item():.4f}')


Loss 1 numpy: 0.4170
Loss 2 numpy: 1.8406


Si queremos obtener las predicciones reales, se puede ahcer lo siguiente:

In [13]:
_,predictions1 = torch.max(Y_pred_good, 1)
_,predictions2 = torch.max(Y_pred_bad, 1)

print(f'predictions1: {predictions1}')
print(f'predictions2: {predictions2}')

predictions1: tensor([0])
predictions2: tensor([1])


Ahi se puede ver cual es la mejor opcion de cada una, asi se obtiene el resultado real de tu prediccion

### Ejemplo con multiples samples


In [16]:
loss = nn.CrossEntropyLoss()
Y = torch.tensor([2,0,1]) # aqui diremos que la clase 2 del primer sample es el mejor, la clase 0 de la segunda es la mejor, etc..
#nsamples x nclasses = 3x3 EN ESTE CASO CAMBIA
Y_pred_good = torch.tensor([[0.1,1.0,2.1],[2.0,1.0,0.1],[0.1,3.0,0.1]])
# Como es una buena prediccion, calza con el de arriba. El primer sample el mayor es el 2.1 de la posicion 2, 
# El segundo el mayor es 2.0 de la posicion 0, etc...
Y_pred_bad = torch.tensor([[2.1,1.0,0.1],[0.1,1.0,2.1],[0.1,3.0,0.1]]) 
# Lo mismo de arriba pero alreves (NO ES el mejor resultado
# Aunque por ejemplo el tercer sample si es una buena prediccion, los otros dos no

l1 = loss(Y_pred_good,Y)
l2 = loss(Y_pred_bad,Y)

print(f'Loss 1 numpy: {l1.item():.4f}')
print(f'Loss 2 numpy: {l2.item():.4f}')

_,predictions1 = torch.max(Y_pred_good, 1)
_,predictions2 = torch.max(Y_pred_bad, 1)

print(f'predictions1: {predictions1}')
print(f'predictions2: {predictions2}')

Loss 1 numpy: 0.3018
Loss 2 numpy: 1.6242
predictions1: tensor([2, 0, 1])
predictions2: tensor([0, 2, 1])


Se puede ver como la primera prediccion es buena y acierta la prediccion, mientras que la segunda prediccion es mala pues tiene alta entropia cruzada y se equivoca en la prediccion.